<a href="https://colab.research.google.com/github/anelsamadulla/energy_consumption/blob/main/%D0%9E%D1%81%D0%B2%D0%B5%D1%89%D0%B5%D0%BD%D0%BD%D0%BE%D1%81%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [10]:
files = {
    "Освещенность_139": "sample_data/Освещенность_139.csv",
    "Освещенность_3": "sample_data/Освещенность_3.csv",
    "Освещенность_6": "sample_data/Освещенность_61.csv",
    "Освещенность_81": "sample_data/Освещенность_81.csv",
}

In [11]:
df1 = pd.read_csv("sample_data/Освещенность_139.csv")
print(df1.columns.tolist())
df2 = pd.read_csv("sample_data/Освещенность_3.csv")
print(df2.columns.tolist())
df3 = pd.read_csv("sample_data/Освещенность_61.csv")
print(df3.columns.tolist())
df4 = pd.read_csv("sample_data/Освещенность_81.csv")
print(df4.columns.tolist())

['Timestamp;criticalAlarmsCount;humidity;lux;majorAlarmsCount;temperature']
['Timestamp;criticalAlarmsCount;humidity;lux;majorAlarmsCount;temperature']
['Timestamp;criticalAlarmsCount;humidity;lux;majorAlarmsCount;temperature']
['Timestamp;criticalAlarmsCount;humidity;lux;majorAlarmsCount;temperature']


In [14]:
LUX_THRESHOLD = 10
START_OFF = 18
END_OFF = 6

summary_list = []

for name, path in files.items():
    df = pd.read_csv(path, sep=";")   # ✅ ВАЖНО
    df.columns = df.columns.str.strip()

    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values("Timestamp").reset_index(drop=True)

    hour = df["Timestamp"].dt.hour
    df["is_off_hours"] = (hour >= START_OFF) | (hour < END_OFF)

    df["light_on"] = df["lux"] > LUX_THRESHOLD
    df["is_violation"] = df["is_off_hours"] & df["light_on"]

    df["dt_hours"] = df["Timestamp"].diff().dt.total_seconds().div(3600).fillna(0)

    violations = df[df["is_violation"]]

    total_hours = violations["dt_hours"].sum()

    summary_list.append({
        "sensor": name,
        "total_violation_hours": total_hours,
        "total_violation_minutes": total_hours * 60
    })

summary = pd.DataFrame(summary_list).sort_values(
    "total_violation_minutes", ascending=False
)

summary

,sensor,total_violation_hours,total_violation_minutes
3,Освещенность_81,111.176667,6670.600000
2,Освещенность_6,70.188889,4211.333333
0,Освещенность_139,23.854444,1431.266667
1,Освещенность_3,10.066667,604.000000
